# Деревья Меркла и Merkle Proofs

Этот notebook охватывает:
- **Часть 1:** Построение дерева Меркла с нуля (CRYPTO-13)
- **Часть 2:** Merkle Proofs -- генерация и верификация (CRYPTO-14)
- **Часть 3:** Свойство commitment -- изменение листа меняет корень
- **Часть 4:** Практическое применение -- Bitcoin блок и airdrop
- **Упражнения и челлендж**

---
## Часть 1: Построение дерева Меркла (CRYPTO-13)

### 1.1 Хеш-функция для дерева

In [ ]:
import hashlib
import os
import time

def sha256(data: bytes) -> bytes:
    """SHA-256 хеш. Возвращает 32 байта."""
    return hashlib.sha256(data).digest()

def sha256_hex(data: bytes) -> str:
    """SHA-256 хеш в hex формате."""
    return hashlib.sha256(data).hexdigest()

# Проверка
h = sha256(b"Hello, Merkle!")
print(f"sha256(b'Hello, Merkle!') = {h.hex()[:16]}...")
print(f"Длина: {len(h)} байт = {len(h) * 8} бит")

### 1.2 Построение дерева Меркла

Алгоритм:
1. Хешируем каждый элемент данных -> получаем листья
2. Попарно конкатенируем и хешируем -> следующий уровень
3. При нечётном количестве -- дублируем последний (конвенция Bitcoin)
4. Повторяем до единственного корня (Merkle Root)

In [ ]:
def build_merkle_tree(leaves: list[bytes]) -> list[list[bytes]]:
    """Строит дерево Меркла.
    
    Args:
        leaves: Список хешей листьев.
    
    Returns:
        Список уровней от листьев (индекс 0) до корня (последний).
        Каждый уровень -- список байтовых хешей.
    """
    if not leaves:
        return []
    
    tree = [leaves[:]]  # Копия листьев
    level = leaves[:]
    
    while len(level) > 1:
        # Дублируем последний при нечётном количестве
        if len(level) % 2 == 1:
            level.append(level[-1])
        
        next_level = []
        for i in range(0, len(level), 2):
            parent = sha256(level[i] + level[i + 1])
            next_level.append(parent)
        
        tree.append(next_level)
        level = next_level
    
    return tree

print("build_merkle_tree загружена!")

### 1.3 Дерево из 4 транзакций

In [ ]:
# 4 транзакции
transactions_4 = [
    b"Alice -> Bob: 1 BTC",
    b"Bob -> Carol: 0.5 BTC",
    b"Dave -> Eve: 2 BTC",
    b"Eve -> Alice: 0.1 BTC",
]

# Хешируем транзакции -> листья
leaves_4 = [sha256(tx) for tx in transactions_4]

# Строим дерево
tree_4 = build_merkle_tree(leaves_4)

# Выводим все уровни
print(f"Количество листьев: {len(tree_4[0])}")
print(f"Количество уровней: {len(tree_4)}")
print()

for lvl_idx, level in enumerate(tree_4):
    level_name = "Листья" if lvl_idx == 0 else (
        "Корень" if lvl_idx == len(tree_4) - 1 else f"Уровень {lvl_idx}"
    )
    print(f"{level_name} ({len(level)} узлов):")
    for i, node in enumerate(level):
        print(f"  [{i}] {node.hex()[:16]}...")
    print()

root_4 = tree_4[-1][0]
print(f"Merkle Root: {root_4.hex()}")

### 1.4 Дерево из 8 транзакций

In [ ]:
transactions_8 = [f"tx_{i}: перевод {i * 0.5:.1f} BTC".encode() for i in range(8)]

leaves_8 = [sha256(tx) for tx in transactions_8]
tree_8 = build_merkle_tree(leaves_8)

print(f"Листьев: {len(tree_8[0])}")
print(f"Уровней: {len(tree_8)}")
print(f"Всего узлов: {sum(len(lvl) for lvl in tree_8)}")
print(f"Merkle Root: {tree_8[-1][0].hex()[:16]}...")

# Проверим формулу: высота = ceil(log2(n))
import math
n = len(leaves_8)
expected_height = math.ceil(math.log2(n))
actual_height = len(tree_8) - 1  # -1 потому что листья -- уровень 0
print(f"\nВысота: {actual_height} (ожидание: ceil(log2({n})) = {expected_height})")
print(f"Всего узлов: {sum(len(lvl) for lvl in tree_8)} (ожидание: 2n-1 = {2*n - 1})")

### 1.5 Обработка нечётного числа листьев

In [ ]:
# 5 транзакций -- нечётное число
transactions_5 = [f"tx_{i}".encode() for i in range(5)]
leaves_5 = [sha256(tx) for tx in transactions_5]
tree_5 = build_merkle_tree(leaves_5)

print(f"Листьев: {len(leaves_5)}")
print(f"Уровней: {len(tree_5)}")

# Показываем, что пятый лист дублируется
print(f"\nУровень 0 (листья): {len(tree_5[0])} узлов")
print(f"Уровень 1: {len(tree_5[1])} узлов")
print(f"  -> При 5 листьях, 5-й дублируется: пара (leaf4, leaf4)")
print(f"Уровень 2: {len(tree_5[2])} узлов")
print(f"Merkle Root: {tree_5[-1][0].hex()[:16]}...")

---
## Часть 2: Merkle Proofs (CRYPTO-14)

### 2.1 Генерация Merkle Proof

In [ ]:
def get_merkle_proof(
    tree: list[list[bytes]],
    leaf_index: int
) -> list[tuple[bytes, str]]:
    """Генерирует Merkle proof для листа.
    
    Args:
        tree: Список уровней дерева (от листьев к корню).
        leaf_index: Индекс листа (0-based).
    
    Returns:
        Список пар (sibling_hash, direction).
        direction = 'right' если sibling справа, 'left' если слева.
    """
    proof = []
    idx = leaf_index
    
    for level in tree[:-1]:  # Все уровни кроме корня
        if idx % 2 == 0:
            sibling_idx = idx + 1
            direction = 'right'
        else:
            sibling_idx = idx - 1
            direction = 'left'
        
        if sibling_idx < len(level):
            proof.append((level[sibling_idx], direction))
        
        idx //= 2  # Переходим к индексу родителя
    
    return proof

print("get_merkle_proof загружена!")

### 2.2 Верификация Merkle Proof

In [ ]:
def verify_merkle_proof(
    leaf_hash: bytes,
    proof: list[tuple[bytes, str]],
    root: bytes
) -> bool:
    """Верифицирует Merkle proof.
    
    Args:
        leaf_hash: Хеш проверяемого листа.
        proof: Список (sibling_hash, direction) пар.
        root: Известный Merkle Root.
    
    Returns:
        True если proof валиден.
    """
    current = leaf_hash
    
    for sibling, direction in proof:
        if direction == 'right':
            current = sha256(current + sibling)
        else:
            current = sha256(sibling + current)
    
    return current == root

print("verify_merkle_proof загружена!")

### 2.3 Генерация и верификация proof

In [ ]:
# Используем дерево из 8 транзакций
root_8 = tree_8[-1][0]

# Генерируем proof для tx_3 (индекс 3)
target_idx = 3
proof = get_merkle_proof(tree_8, target_idx)

print(f"Proof для листа [{target_idx}] ({transactions_8[target_idx]}):")
print(f"Количество элементов: {len(proof)} (log2(8) = {math.ceil(math.log2(8))})")
print()

for i, (sibling, direction) in enumerate(proof):
    print(f"  Элемент {i}: ({sibling.hex()[:16]}..., '{direction}')")

# Верифицируем
valid = verify_merkle_proof(leaves_8[target_idx], proof, root_8)
print(f"\nProof валиден: {valid}")

In [ ]:
# Верифицируем proof для КАЖДОГО листа
print("Верификация proofs для всех 8 листьев:")
print("-" * 60)

for i in range(len(leaves_8)):
    proof_i = get_merkle_proof(tree_8, i)
    valid_i = verify_merkle_proof(leaves_8[i], proof_i, root_8)
    status = "OK" if valid_i else "FAIL"
    print(f"  Лист [{i}]: proof = {len(proof_i)} элементов, верификация: {status}")

# Попытка с фейковым листом
fake_leaf = sha256(b"fake_transaction")
proof_2 = get_merkle_proof(tree_8, 2)
fake_valid = verify_merkle_proof(fake_leaf, proof_2, root_8)
print(f"\nФейковый лист с proof для [2]: {fake_valid}  (ожидание: False)")

### 2.4 Размер proof: log2(n)

In [ ]:
# Проверяем размер proof для деревьев разного размера
print(f"{'Листьев':>10} | {'Размер proof':>12} | {'ceil(log2(n))':>13} | {'Совпадает':>9}")
print("-" * 55)

for n in [4, 8, 16, 32, 64, 128, 256, 512, 1024]:
    leaves_n = [sha256(f"tx_{i}".encode()) for i in range(n)]
    tree_n = build_merkle_tree(leaves_n)
    proof_n = get_merkle_proof(tree_n, 0)
    expected = math.ceil(math.log2(n))
    match = "da" if len(proof_n) == expected else "NET"
    print(f"{n:>10} | {len(proof_n):>12} | {expected:>13} | {match:>9}")

---
## Часть 3: Свойство commitment

### 3.1 Изменение одного листа меняет корень

In [ ]:
# Оригинальное дерево
original_txs = [b"tx_0", b"tx_1", b"tx_2", b"tx_3"]
original_leaves = [sha256(tx) for tx in original_txs]
original_tree = build_merkle_tree(original_leaves)
original_root = original_tree[-1][0]

# Изменяем tx_2
modified_txs = [b"tx_0", b"tx_1", b"MODIFIED_tx_2", b"tx_3"]
modified_leaves = [sha256(tx) for tx in modified_txs]
modified_tree = build_merkle_tree(modified_leaves)
modified_root = modified_tree[-1][0]

print("Оригинальное дерево:")
for lvl in range(len(original_tree)):
    for idx in range(len(original_tree[lvl])):
        orig = original_tree[lvl][idx].hex()[:16]
        mod = modified_tree[lvl][idx].hex()[:16]
        changed = " <-- ИЗМЕНЕН" if orig != mod else ""
        print(f"  Уровень {lvl}, узел {idx}: {orig}... {changed}")

print(f"\nОригинальный Merkle Root: {original_root.hex()[:16]}...")
print(f"Изменённый Merkle Root:   {modified_root.hex()[:16]}...")
print(f"Корни совпадают: {original_root == modified_root}")

### 3.2 Попытка создать фейковый proof

In [ ]:
# Можно ли создать proof для транзакции, которой нет в дереве?
fake_tx = sha256(b"fake: steal all BTC")

# Берём proof от настоящего листа [0]
real_proof = get_merkle_proof(original_tree, 0)

# Пытаемся верифицировать фейковый лист с настоящим proof
fake_valid = verify_merkle_proof(fake_tx, real_proof, original_root)
print(f"Фейковый лист + настоящий proof: {fake_valid}")
print("-> Невозможно подделать proof без нахождения коллизии SHA-256!")

# Пытаемся со случайным proof
random_proof = [(os.urandom(32), 'right'), (os.urandom(32), 'left')]
random_valid = verify_merkle_proof(fake_tx, random_proof, original_root)
print(f"\nФейковый лист + случайный proof: {random_valid}")
print("-> Вероятность угадать: 2^(-256) -- практически невозможно")

---
## Часть 4: Практическое применение

### 4.1 Симуляция Bitcoin блока

In [ ]:
# Симулируем блок с 100 транзакциями
num_txs = 100
block_txs = [os.urandom(250) for _ in range(num_txs)]  # ~250 байт каждая
block_leaves = [sha256(tx) for tx in block_txs]
block_tree = build_merkle_tree(block_leaves)
block_root = block_tree[-1][0]

print(f"Блок: {num_txs} транзакций")
print(f"Размер данных блока: ~{num_txs * 250 / 1024:.1f} КБ")
print(f"Merkle Root: {block_root.hex()[:16]}...")
print(f"Высота дерева: {len(block_tree) - 1}")

# Проверяем одну транзакцию
target = 42
proof = get_merkle_proof(block_tree, target)
valid = verify_merkle_proof(block_leaves[target], proof, block_root)

proof_size_bytes = len(proof) * 32  # Каждый хеш = 32 байта
print(f"\nПроверка tx #{target}:")
print(f"  Размер proof: {len(proof)} хешей = {proof_size_bytes} байт")
print(f"  vs полная проверка: {num_txs} хешей = {num_txs * 32} байт")
print(f"  Экономия: {(1 - proof_size_bytes / (num_txs * 32)) * 100:.1f}%")
print(f"  Proof валиден: {valid}")

### 4.2 Airdrop Whitelist

In [ ]:
def create_airdrop_tree(
    recipients: list[tuple[str, int]]
) -> tuple[list[list[bytes]], list[bytes]]:
    """Создаёт Merkle дерево для airdrop whitelist.
    
    Args:
        recipients: Список пар (address, amount).
    
    Returns:
        (tree, leaves) -- дерево и листья.
    """
    # Кодируем: keccak256(abi.encodePacked(address, amount))
    # Для простоты используем SHA-256 вместо Keccak
    leaves = []
    for addr, amount in recipients:
        leaf_data = f"{addr}:{amount}".encode()
        leaves.append(sha256(leaf_data))
    
    tree = build_merkle_tree(leaves)
    return tree, leaves

def check_airdrop_eligibility(
    address: str,
    amount: int,
    tree: list[list[bytes]],
    leaves: list[bytes],
    root: bytes
) -> tuple[bool, list | None]:
    """Проверяет, может ли адрес получить airdrop.
    
    Returns:
        (eligible, proof) -- право на получение и proof.
    """
    leaf_data = f"{address}:{amount}".encode()
    leaf_hash = sha256(leaf_data)
    
    # Ищем лист в дереве
    try:
        idx = leaves.index(leaf_hash)
    except ValueError:
        return False, None
    
    proof = get_merkle_proof(tree, idx)
    valid = verify_merkle_proof(leaf_hash, proof, root)
    return valid, proof

# Создаём whitelist из 20 адресов
whitelist = [
    (f"0x{os.urandom(20).hex()}", (i + 1) * 100)
    for i in range(20)
]

# Добавляем известный адрес
known_address = "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045"  # Пример
known_amount = 1000
whitelist.append((known_address, known_amount))

# Строим Merkle дерево
airdrop_tree, airdrop_leaves = create_airdrop_tree(whitelist)
airdrop_root = airdrop_tree[-1][0]

print(f"Airdrop Whitelist:")
print(f"  Адресов: {len(whitelist)}")
print(f"  Merkle Root: 0x{airdrop_root.hex()[:16]}...")
print(f"  (Этот root хранится в смарт-контракте)")
print()

# Проверяем известный адрес
eligible, proof = check_airdrop_eligibility(
    known_address, known_amount,
    airdrop_tree, airdrop_leaves, airdrop_root
)
print(f"Проверка {known_address[:10]}... на {known_amount} токенов:")
print(f"  Результат: {'Eligible' if eligible else 'Not eligible'}")
if proof:
    print(f"  Proof: {len(proof)} элементов")

# Проверяем неизвестный адрес
unknown_address = "0x0000000000000000000000000000000000000000"
eligible_unknown, _ = check_airdrop_eligibility(
    unknown_address, 9999,
    airdrop_tree, airdrop_leaves, airdrop_root
)
print(f"\nПроверка {unknown_address[:10]}... на 9999 токенов:")
print(f"  Результат: {'Eligible' if eligible_unknown else 'Not eligible'}")

---
## Упражнения

### Упражнение 1: Дерево из 1000 элементов

In [ ]:
# Постройте дерево из 1000 случайных элементов.
# Измерьте: размер proof, время построения, время верификации.

n = 1000
leaves_1k = [sha256(os.urandom(32)) for _ in range(n)]

# Время построения
start = time.time()
tree_1k = build_merkle_tree(leaves_1k)
build_time = time.time() - start

root_1k = tree_1k[-1][0]

# Время генерации proof
start = time.time()
proof_1k = get_merkle_proof(tree_1k, 500)
proof_time = time.time() - start

# Время верификации
start = time.time()
valid_1k = verify_merkle_proof(leaves_1k[500], proof_1k, root_1k)
verify_time = time.time() - start

print(f"Дерево из {n} элементов:")
print(f"  Высота: {len(tree_1k) - 1}")
print(f"  Размер proof: {len(proof_1k)} хешей")
print(f"  Proof валиден: {valid_1k}")
print(f"\nВремя:")
print(f"  Построение дерева:  {build_time * 1000:.2f} мс")
print(f"  Генерация proof:    {proof_time * 1000:.4f} мс")
print(f"  Верификация proof:  {verify_time * 1000:.4f} мс")

### Упражнение 2: Multi-proof (проверка нескольких листьев)

In [ ]:
# Реализуйте multi-proof: проверка нескольких листьев одновременно.
# Подсказка: если два листа -- соседи, их общий sibling не нужно включать дважды.

def get_multi_proof(
    tree: list[list[bytes]],
    leaf_indices: list[int]
) -> list[tuple[bytes, int, str]]:
    """Генерирует multi-proof для нескольких листьев.
    
    Returns:
        Список (hash, level, direction) -- уникальные sibling хеши.
    """
    # TODO: реализуйте самостоятельно
    # Подсказка:
    # 1. Для каждого уровня, определите какие узлы уже "известны" (на пути от листьев)
    # 2. Для каждого известного узла, добавляем sibling только если он НЕ известен
    # 3. Переходим на следующий уровень
    pass

# Проверка (после реализации):
# multi = get_multi_proof(tree_8, [0, 1, 4])
# print(f"Multi-proof для [0,1,4]: {len(multi)} элементов")
# print(f"vs отдельные proofs: {sum(len(get_merkle_proof(tree_8, i)) for i in [0,1,4])} элементов")

### Упражнение 3: Сравнение времени полной проверки vs Merkle Proof

In [ ]:
# Сравните время:
# 1. Полная проверка: пересчитать ВСЁ дерево и проверить root
# 2. Merkle Proof: верифицировать только proof path

sizes = [100, 1000, 10_000, 100_000]

print(f"{'N':>10} | {'Полная (мс)':>12} | {'Proof (мс)':>12} | {'Ускорение':>10}")
print("-" * 55)

for n in sizes:
    leaves_n = [sha256(f"tx_{i}".encode()) for i in range(n)]
    tree_n = build_merkle_tree(leaves_n)
    root_n = tree_n[-1][0]
    
    # Полная проверка: перестроить дерево
    start = time.time()
    for _ in range(10):  # 10 повторений для точности
        rebuilt = build_merkle_tree(leaves_n)
        assert rebuilt[-1][0] == root_n
    full_time = (time.time() - start) / 10 * 1000
    
    # Merkle Proof
    proof_n = get_merkle_proof(tree_n, n // 2)
    start = time.time()
    for _ in range(10000):  # Больше повторений т.к. очень быстро
        verify_merkle_proof(leaves_n[n // 2], proof_n, root_n)
    proof_time = (time.time() - start) / 10000 * 1000
    
    speedup = full_time / proof_time if proof_time > 0 else float('inf')
    print(f"{n:>10} | {full_time:>12.3f} | {proof_time:>12.6f} | {speedup:>9.0f}x")

---
## Челлендж: Sparse Merkle Tree (SMT)

Обычное дерево Меркла доказывает **включение** (данные есть в дереве).
Sparse Merkle Tree также может доказать **отсутствие** (данных НЕТ в дереве).

Принцип:
- Фиксированная глубина (например, 256 для SHA-256 ключей)
- Большинство листьев = пустые (нулевой хеш)
- Ключ определяет позицию листа (каждый бит ключа = left/right)
- Для доказательства отсутствия: показываем, что лист по данному ключу пуст

Реализуйте простой SMT для key-value хранилища.

In [ ]:
# Подсказка: начните с малой глубины (например, 8 бит = 256 возможных ключей)

EMPTY_HASH = b'\x00' * 32  # Хеш пустого листа

class SparseMerkleTree:
    """Sparse Merkle Tree для key-value доказательств."""
    
    def __init__(self, depth: int = 8):
        self.depth = depth
        self.data: dict[int, bytes] = {}  # key -> value_hash
        # Предвычисленные хеши пустых поддеревьев
        self.empty_hashes = [EMPTY_HASH]
        for _ in range(depth):
            prev = self.empty_hashes[-1]
            self.empty_hashes.append(sha256(prev + prev))
    
    def insert(self, key: int, value: bytes):
        """Вставляет значение по ключу."""
        # TODO: реализуйте
        pass
    
    def get_root(self) -> bytes:
        """Возвращает корень дерева."""
        # TODO: реализуйте
        pass
    
    def get_proof(self, key: int) -> list[tuple[bytes, str]]:
        """Генерирует proof включения ИЛИ отсутствия."""
        # TODO: реализуйте
        pass
    
    def verify(self, key: int, value: bytes | None, proof: list, root: bytes) -> bool:
        """Верифицирует proof включения (value != None) или отсутствия (value == None)."""
        # TODO: реализуйте
        pass

# Тесты (после реализации):
# smt = SparseMerkleTree(depth=8)
# smt.insert(42, b"hello")
# smt.insert(100, b"world")
# root = smt.get_root()
# 
# # Proof включения
# proof_42 = smt.get_proof(42)
# assert smt.verify(42, b"hello", proof_42, root)
# 
# # Proof отсутствия
# proof_7 = smt.get_proof(7)
# assert smt.verify(7, None, proof_7, root)